# 测试softmax是怎么使用的

In [7]:
from keras.models import *
from keras.layers import *
import keras.losses as loss
import numpy as np
import warnings
warnings.filterwarnings("ignore")

N_SAMPLES = 100
N_EPOCHES = 10
DIM_INPUT = 100
DIM_OUTPUT = 10
BATCH_SIZE = 32

## 生成数据，构建模型

In [5]:
print('generate data...')
x = np.random.randint(10, size=[N_SAMPLES, DIM_INPUT])
y_indices = np.random.randint(10, size=N_SAMPLES)
onehots = []
for i in range(DIM_OUTPUT):
    line = []
    for j in range(DIM_OUTPUT):
        if j == i:
            line.append(1)
        else:
            line.append(0)
    onehots.append(line)
onehots = np.array(onehots)
y = onehots[y_indices]

print('x[:4]', x[:4])
print('y[:4]', y[:4])

generate data...
x[:4] [[2 0 6 0 1 2 2 1 4 4 9 1 3 1 7 0 2 2 6 5 4 5 9 5 8 0 1 3 4 9 8 0 4 2 6 8
  6 4 5 0 0 6 4 4 1 3 7 8 8 6 3 2 5 3 3 0 2 4 0 4 8 3 6 4 0 4 4 2 6 4 3 9
  7 4 6 6 0 8 4 6 7 5 3 7 4 4 7 2 7 8 9 5 4 0 6 5 5 8 0 0]
 [5 8 2 3 9 5 7 3 7 5 9 9 4 9 1 4 8 4 8 8 7 5 9 4 3 3 0 7 4 7 7 4 6 7 3 1
  1 4 2 6 1 4 5 6 5 7 7 3 2 3 0 0 6 1 8 8 3 3 5 2 9 1 9 4 0 9 1 6 2 2 2 5
  4 1 1 2 8 2 6 8 4 9 8 7 3 9 3 0 4 3 2 9 6 9 6 9 0 4 4 5]
 [6 6 9 0 8 3 2 8 2 5 6 4 0 9 8 7 3 9 7 7 2 2 4 7 9 8 3 4 0 7 1 4 2 1 5 8
  4 4 6 9 7 8 5 0 8 5 6 2 7 0 3 1 3 7 4 7 4 7 7 1 7 7 4 9 3 4 3 6 4 6 7 3
  3 7 8 9 4 6 7 9 6 0 7 9 2 4 1 5 2 9 1 6 9 1 7 5 5 5 3 9]
 [2 8 1 9 0 2 2 3 1 7 9 6 9 3 7 9 9 9 2 9 7 6 5 3 0 8 5 9 6 0 2 8 3 7 1 7
  4 4 1 1 3 8 0 5 7 7 9 0 5 8 6 7 2 3 1 8 3 5 5 3 7 7 5 3 6 9 7 6 4 7 9 0
  0 2 8 2 9 3 7 9 2 4 6 0 4 0 7 9 0 3 5 1 0 6 8 7 9 8 8 8]]
y[:4] [[0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0]]


In [8]:
print('build model...')
input = Input(batch_shape=[None, DIM_INPUT])             # INPUT = BATCH_SIZE x DIM_INPUT
interme = Dense(64, name='interme')(input)
output = Dense(DIM_OUTPUT, activation='softmax')(interme)# W = DIM_INPUT x DIM_OUTPUT, b = DIM_OUTPUT
model = Model(inputs=[input], output=[output], )         # OUTPUT = BATCH_SIZE x DIM_OUTPUT
model.compile('adam', loss=loss.binary_crossentropy, metrics=['acc'])


build model...


## 验证softmax是否是对最后一维归一化

In [10]:
print('softmax result check...')
last_layer = Model(input=model.input, output=model.output)
result_layer = last_layer.predict(x)
last_sample_output = result_layer[-1]
sum = 0
for i in last_sample_output:
    print('%.2f' % i)
    sum += i
print('sum:', sum)

softmax result check...
0.01
0.00
0.00
0.00
0.97
0.00
0.02
0.00
0.00
0.00
sum: 1.0000000121989143


In [11]:
print('interme layer check...')
last_layer = Model(input=model.input, output=model.get_layer('interme').output)
intermediate_output = last_layer.predict(x)
print(intermediate_output.shape)

interme layer check...
(100, 64)


## 结果
softmax对最后一维归一化，并且有一定程度的精度损失